인터넷에 있는 자료중에 Excel 형태나 CSV형태로 파일을 다운로드 받는다.
KRX(한국증권거래소)사이트에 있는 관리종목의 시장구분에 따른 파일을 Excel, CSV형태로 다운받는 크롤러 제작한다.
URL : http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020301
 실행후  시장구분선택 -> 조회 -> Excel/CSV 파일로 저장 순서 진행한다.

Chrome driver에서 다운로드 설정
Chrome driver 인스턴스 생성시 다운로드관련 옵션이 필요하다

크롬 드라이버는 악의적인 행동을 예방하기 위해 소프트웨어가 컴퓨터에서 파일을 다운로드 하지 못하도록 한다.
이를 해결하기 위해 크롬 커맨드라인에 다운로드를 허용하는 명령을 추가해야 한다.

In [1]:
#step1. 환경설정 (라이브러리 import) 
print('='*80)
print('예제: 파일 다운로드용 크롤러 만들기 - xlsx 와 csv 형식 다운로드')
print('='*80)

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os

예제: 파일 다운로드용 크롤러 만들기 - xlsx 와 csv 형식 다운로드


In [2]:
#Step 2. 사용자에게 주소를 입력 받습니다. 다운 받을 폴더도 생성
url_add = input('1. 파일을 다운로드 할 웹페이지 주소를 입력:')
f_dir = input("2.파일이 저장 될 폴더경로를 입력:(C:\\Study\\Python\\notebook\\data\\)")

if os.path.isdir(f_dir):
    print('입력하신 경로가 존재하여  %s 폴더에 저장하겠습니다' %f_dir)
else:
    os.makedirs(f_dir)
    print('입력하신 경로가 존재하지 않아 %s 폴더를 생성했습니다' %f_dir)
    
#어떤 형식으로 받을지 지정
choice = input('''1)전체    2) KOSPI    3) KOSDAQ    4)KONEX    3.위번호 중 조회 할 시장 번호를 선택하세요 ''')

f_choice =  input(''' 1.xlsx 형식으로 저장하기      2.csv 형식으로 저장하기  
4. 위 번호 중 저장할 파일 형식의 번호를 선택하세요:  ''')

print("\n")
print("요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~^^")

1. 파일을 다운로드 할 웹페이지 주소를 입력:http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020301
2.파일이 저장 될 폴더경로를 입력:(C:\Study\Python\notebook\data\)C:\Study\Python\notebook\data\
입력하신 경로가 존재하여  C:\Study\Python\notebook\data\ 폴더에 저장하겠습니다
1)전체    2) KOSPI    3) KOSDAQ    4)KONEX    3.위번호 중 조회 할 시장 번호를 선택하세요 2
 1.xlsx 형식으로 저장하기      2.csv 형식으로 저장하기  
4. 위 번호 중 저장할 파일 형식의 번호를 선택하세요:  1


요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~^^


In [6]:
#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )
 
options = webdriver.ChromeOptions()
 
path = "C:\\Study\\Python\\datadown\\chromedriver.exe"
driver = webdriver.Chrome(path,options=options)
 
#다운로드 될 파일의 저장폴더지정,
##propmt 창이 뜨지 않는다.
#pdf형식이 다운로드되면 브라우저에서 보이지 않고 무조건 다운로드된다.
options.add_experimental_option("prefs", {
      "download.default_directory": f_dir,
      "download.prompt_for_download": False,
      "plugins.always_open_pdf_externally": True 
})
 
#크롬 드라이버는 악의적인 행동을 예방하기 위해 소프트웨어가 컴퓨터에서 파일을 다운로드 하지 못하도록 한다.
#이를 해결하기 위해 크롬 커맨드라인에 다운로드를 허용하는 명령을 추가해야 한다.(
 
driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': f_dir}}
command_result = driver.execute("send_command", params)
 
driver.get(url_add)
time.sleep(10)


In [7]:
driver.maximize_window()

In [8]:
# chrome driver로 열린 브라우저의 개발자 도구에서 id를 새롭게 복사해온다
# Angular로 작성된 요소에서 click ()호출 위해 WebDriverWait에서 element_to_be_clickable ()사용
 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [9]:
#Step 4. 전체 / KOSPI / KOSDAQ /KONEx 라디오 버튼 선택 후 조회 버튼 누르기
if choice == '1':
    WebDriverWaiter(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='mktld_0_0']"))).click()
    driver.find.element_by_id("jsSearchButton").click() #검색버튼
elif choice == '2' :
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='mktId_0_1']"))).click()
    driver.find_element_by_id("jsSearchButton").click()#검색버튼
elif choice =='3' :
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='mktId_0_2']"))).click()
    driver.find_element_by_id("jsSearchButton").click()#검색버튼
elif choice =='4' :
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='mktId_0_3']"))).click()
    driver.find_element_by_id("jsSearchButton").click()#검색버튼
else :
    print('번호를 다시 확인해 주세요')   

In [11]:
# chrome driver로 열린 브라우저의 개발자 도구에서 xpath를 새롭게 복사해온다
# 이부분을 먼저 실행해서 xlsx와 csv형태를 선택하는 팝업창을 연다.
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="MDCSTAT022_FORM"]/div[2]/div/p[2]/button[2]/img'))).click()
 
# chrome driver로 열린 브라우저의 개발자 도구에서 xpath를 새롭게 복사해온다
# 팝업창의 위치를 변경하면 xpath값도 변경되므로 열려진 채로 그대로 작업한다.

#Step 5. xls 형태와 csv 형태의 파일로 다운로드 받기                                   
if f_choice == '1' :
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ui-id-1"]/div/div[1]/a'))).click()                                   
elif choice == '2' :
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ui-id-1"]/div/div[2]/a'))).click()
    
time.sleep(10)

e_time = time.time( )
t_time = e_time - s_time
 
#Step 6. 요약 정보 보여주기
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일이 저장된 폴더명 : %s " %f_dir)
print("=" *100)
 
driver.close( )

총 소요시간은 631.7 초 입니다 
파일이 저장된 폴더명 : C:\Study\Python\notebook\data\ 
